In [ ]:
from scipy.io import loadmat

# مسیر فایل mat
data = loadmat('/content/bcdr_data_whole.mat')

# نمایش محتوای دیکشنری
print(data)


{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon May 25 23:33:54 2020', '__version__': '1.0', '__globals__': [], 'X': array([[2.17372803e+02, 2.26000000e+02, 3.34304314e+01, ...,
        4.09711838e+01, 5.43651000e-04, 7.60290289e+00],
       [1.34569305e+02, 1.32000000e+02, 1.64031601e+01, ...,
        5.33333321e+01, 2.89329200e-03, 6.81921005e+00],
       [1.29835617e+02, 1.31000000e+02, 1.75003757e+01, ...,
        1.62746525e+01, 1.58815900e-03, 7.04570007e+00],
       ...,
       [6.03789749e+01, 6.10000000e+01, 6.78357172e+00, ...,
        5.33712101e+00, 1.20207380e-02, 5.62138128e+00],
       [1.17874023e+02, 1.08000000e+02, 4.94650116e+01, ...,
        1.67071896e+01, 3.75119000e-04, 7.84101486e+00],
       [1.94227890e+02, 1.94000000e+02, 3.78509216e+01, ...,
        4.87280769e+01, 3.98454000e-04, 8.16876030e+00]]), 'Y': array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],

In [ ]:
len(data['X'][0])

17

In [ ]:
y=data['Y']
x=data['X']

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 📥 1. بارگذاری دیتاست
data = load_breast_cancer()

# 🎯 ویژگی‌ها و برچسب‌ها
X = data.data
y = data.target  # 0: بدخیم (malignant), 1: خوش‌خیم (benign)

# 🔍 نمایش اطلاعات کلی
#print("ویژگی‌ها:", data.feature_names)
#print("توضیحات:", data.DESCR[:500], '...')  # فقط ۵۰۰ کاراکتر اول
#print("شکل داده‌ها:", X.shape)
#print("برچسب‌ها:", set(y))

# 🧪 2. تقسیم داده‌ها
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔄 3. نرمال‌سازی ویژگی‌ها
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# ✅ آماده‌ی استفاده برای مدل‌های ML یا deep learning


In [ ]:
from scipy.io import loadmat
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score
import numpy as np

# 💾 بارگذاری فایل .mat
#data = loadmat('/content/bcdr_data_whole.mat')
#X = data['X']
#y = data['Y']

# 🔧 اصلاح شکل y
#y = y.squeeze()  # حالا y شکلش (n,) یا (n,1) خواهد بود

# 🧪 تقسیم داده
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔄 نرمال‌سازی
scaler = StandardScaler()
X_trainval = scaler.fit_transform(X_trainval)
X_test = scaler.transform(X_test)

# 🔻 کاهش بعد با PCA
pca = PCA(n_components=0.95)
pca.fit(X_trainval)
bottleneck_dim = pca.n_components_
print(f"Bottleneck dim from PCA: {bottleneck_dim}")

# 🧠 تعریف انکودر و کلاسیفایر
class Encoder(nn.Module):
    def __init__(self, input_dim, bottleneck_dim):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, bottleneck_dim)
        )

    def forward(self, x):
        return self.encoder(x)

class Classifier(nn.Module):
    def __init__(self, bottleneck_dim):
        super(Classifier, self).__init__()
        self.classifier = nn.Linear(bottleneck_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.classifier(x))

# 🚀 تابع آموزش و ارزیابی
def train_and_evaluate(X_trainval, y_trainval, X_test, y_test, bottleneck_dim, device):
    # تبدیل به تنسور
    X_trainval_tensor = torch.tensor(X_trainval, dtype=torch.float32).to(device)
    y_trainval_tensor = torch.tensor(y_trainval.squeeze(), dtype=torch.float32).unsqueeze(1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.squeeze(), dtype=torch.int32)

    encoder = Encoder(input_dim=X_trainval.shape[1], bottleneck_dim=bottleneck_dim).to(device)
    classifier = Classifier(bottleneck_dim=bottleneck_dim).to(device)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()), lr=1e-3)

    epochs = 20
    for epoch in range(epochs):
        encoder.train()
        classifier.train()
        optimizer.zero_grad()
        z = encoder(X_trainval_tensor)
        y_pred = classifier(z)
        loss = criterion(y_pred, y_trainval_tensor)
        loss.backward()
        optimizer.step()

    # ارزیابی روی تست
    encoder.eval()
    classifier.eval()
    with torch.no_grad():
        z_test = encoder(X_test_tensor)
        y_test_pred_prob = classifier(z_test).cpu().numpy().flatten()
        y_test_pred_labels = (y_test_pred_prob >= 0.5).astype(int)

    # محاسبه معیارها
    accuracy = (y_test_pred_labels == y_test_tensor.numpy()).mean()
    f1 = f1_score(y_test_tensor, y_test_pred_labels, zero_division=0)
    precision = precision_score(y_test_tensor, y_test_pred_labels, zero_division=0)
    recall = recall_score(y_test_tensor, y_test_pred_labels, zero_division=0)
    try:
        auc = roc_auc_score(y_test_tensor, y_test_pred_prob)
    except ValueError:
        auc = 0.5

    return accuracy, f1, auc, precision, recall, y_test_pred_prob

# ⚡ اجرای چندباره
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_runs = 10
accuracies, f1_scores, aucs, precisions, recalls = [], [], [], [], []

for run in range(num_runs):
    acc, f1, auc, prec, rec, y_test_pred_prob = train_and_evaluate(X_trainval, y_trainval, X_test, y_test, bottleneck_dim, device)
    print(f"Run {run+1}: Acc={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}, Prec={prec:.4f}, Recall={rec:.4f}")
    accuracies.append(acc)
    f1_scores.append(f1)
    aucs.append(auc)
    precisions.append(prec)
    recalls.append(rec)

print("\n=== Summary over runs ===")
print(f"Mean Accuracy:  {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Mean F1:       {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"Mean AUC:      {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"Mean Precision:{np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Mean Recall:   {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")


Bottleneck dim from PCA: 10
Run 1: Acc=0.9561, F1=0.9645, AUC=0.9951, Prec=0.9714, Recall=0.9577
Run 2: Acc=0.9561, F1=0.9645, AUC=0.9866, Prec=0.9714, Recall=0.9577
Run 3: Acc=0.9561, F1=0.9645, AUC=0.9925, Prec=0.9714, Recall=0.9577
Run 4: Acc=0.9561, F1=0.9645, AUC=0.9921, Prec=0.9714, Recall=0.9577
Run 5: Acc=0.9561, F1=0.9645, AUC=0.9836, Prec=0.9714, Recall=0.9577
Run 6: Acc=0.9561, F1=0.9645, AUC=0.9961, Prec=0.9714, Recall=0.9577
Run 7: Acc=0.9561, F1=0.9645, AUC=0.9918, Prec=0.9714, Recall=0.9577
Run 8: Acc=0.9561, F1=0.9645, AUC=0.9872, Prec=0.9714, Recall=0.9577
Run 9: Acc=0.9561, F1=0.9645, AUC=0.9951, Prec=0.9714, Recall=0.9577
Run 10: Acc=0.9561, F1=0.9645, AUC=0.9912, Prec=0.9714, Recall=0.9577

=== Summary over runs ===
Mean Accuracy:  0.9561 ± 0.0000
Mean F1:       0.9645 ± 0.0000
Mean AUC:      0.9911 ± 0.0039
Mean Precision:0.9714 ± 0.0000
Mean Recall:   0.9577 ± 0.0000


In [ ]:
y_test_pred_labels = (y_test_pred_prob >= 0.8).astype(int)
accuracy = (y_test_pred_labels == y_test).mean()
accuracy

np.float64(0.8333333333333334)

In [ ]:
7!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import numpy as np
import optuna
import warnings

warnings.filterwarnings("ignore")

# برای reproducibility
#torch.manual_seed(42)
#np.random.seed(42)

# ✅ 1) داده ساختگی
X, y = make_classification(n_samples=50, n_features=500, n_classes=2, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# تقسیم به train+val و test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ 2) تخمین اولیه bottleneck_dim با PCA
pca = PCA(n_components=0.95)  # 95% واریانس
pca.fit(X_trainval)
initial_bottleneck = pca.n_components_
print(f"Initial bottleneck_dim from PCA: {initial_bottleneck}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ✅ 3) تعریف مدل‌ها
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, bottleneck_dim):
        super(Encoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, bottleneck_dim)
        )
    def forward(self, x):
        return self.net(x)

class Classifier(nn.Module):
    def __init__(self, bottleneck_dim):
        super(Classifier, self).__init__()
        self.clf = nn.Linear(bottleneck_dim, 1)
    def forward(self, x):
        return torch.sigmoid(self.clf(x))


# ✅ 4) تابع آموزش و ارزیابی
def train_and_evaluate(X_trainval, y_trainval, X_test, y_test, bottleneck_dim, hidden_dim, lr, device):
    X_trainval_tensor = torch.tensor(X_trainval, dtype=torch.float32).to(device)
    y_trainval_tensor = torch.tensor(y_trainval, dtype=torch.float32).unsqueeze(1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

    encoder = Encoder(input_dim=X_trainval.shape[1], hidden_dim=hidden_dim, bottleneck_dim=bottleneck_dim).to(device)
    classifier = Classifier(bottleneck_dim=bottleneck_dim).to(device)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()), lr=lr)

    epochs = 30
    encoder.train()
    classifier.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        z = encoder(X_trainval_tensor)
        y_pred = classifier(z)
        loss = criterion(y_pred, y_trainval_tensor)
        loss.backward()
        optimizer.step()

    # ارزیابی روی تست
    encoder.eval()
    classifier.eval()
    with torch.no_grad():
        z_test = encoder(X_test_tensor)
        y_test_pred = classifier(z_test).cpu().numpy()
        y_test_labels = (y_test_pred >= 0.5).astype(int)

        acc = accuracy_score(y_test, y_test_labels)
        f1 = f1_score(y_test, y_test_labels)
        auc = roc_auc_score(y_test, y_test_pred)
        prec = precision_score(y_test, y_test_labels)
        rec = recall_score(y_test, y_test_labels)
    return acc, f1, auc, prec, rec


# ✅ 5) تابع هدف برای Optuna
def objective(trial):
    bottleneck_dim = trial.suggest_int('bottleneck_dim', 10, initial_bottleneck)
    hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)

    val_f1_scores = []
    for run in range(3):  # برای پایداری چند بار تکرار
        acc, f1, auc, prec, rec = train_and_evaluate(
            X_trainval, y_trainval, X_test, y_test, bottleneck_dim, hidden_dim, lr, device
        )
        val_f1_scores.append(f1)

    mean_f1 = np.mean(val_f1_scores)
    return mean_f1  # هدف: بیشینه کردن F1


# ✅ 6) اجرای optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print(f"\n✅ بهترین پارامترها:")
print(study.best_params)

# ✅ 7) آموزش نهایی با بهترین پارامتر و ارزیابی روی test
best_params = study.best_params
final_acc, final_f1, final_auc, final_prec, final_rec = train_and_evaluate(
    X_trainval, y_trainval, X_test, y_test,
    bottleneck_dim=best_params['bottleneck_dim'],
    hidden_dim=best_params['hidden_dim'],
    lr=best_params['lr'],
    device=device
)

print("\n📊 Final Test Results:")
print(f"Accuracy: {final_acc:.4f}")
print(f"F1-score: {final_f1:.4f}")
print(f"AUC: {final_auc:.4f}")
print(f"Precision: {final_prec:.4f}")
print(f"Recall (Sensitivity): {final_rec:.4f}")
print(f"Specificity: {1 - final_rec:.4f}")  # برای تست: 1 - recall در داده‌های منفی


[I 2025-07-03 15:40:28,462] A new study created in memory with name: no-name-7fa0e324-b782-4cb4-9477-fec0bf6cc3b2


Initial bottleneck_dim from PCA: 36


[I 2025-07-03 15:40:28,923] Trial 0 finished with value: 0.6222222222222222 and parameters: {'bottleneck_dim': 35, 'hidden_dim': 78, 'lr': 0.0004947407944969044}. Best is trial 0 with value: 0.6222222222222222.
[I 2025-07-03 15:40:29,205] Trial 1 finished with value: 0.7167277167277167 and parameters: {'bottleneck_dim': 12, 'hidden_dim': 35, 'lr': 0.003833725566317183}. Best is trial 1 with value: 0.7167277167277167.
[I 2025-07-03 15:40:29,670] Trial 2 finished with value: 0.6444444444444444 and parameters: {'bottleneck_dim': 23, 'hidden_dim': 161, 'lr': 0.0003883861967460078}. Best is trial 1 with value: 0.7167277167277167.
[I 2025-07-03 15:40:30,042] Trial 3 finished with value: 0.6262626262626262 and parameters: {'bottleneck_dim': 11, 'hidden_dim': 52, 'lr': 0.00973128001236248}. Best is trial 1 with value: 0.7167277167277167.
[I 2025-07-03 15:40:30,417] Trial 4 finished with value: 0.6655011655011654 and parameters: {'bottleneck_dim': 29, 'hidden_dim': 78, 'lr': 0.00020497456941332


✅ بهترین پارامترها:
{'bottleneck_dim': 12, 'hidden_dim': 35, 'lr': 0.003833725566317183}

📊 Final Test Results:
Accuracy: 0.7000
F1-score: 0.7273
AUC: 0.8000
Precision: 0.6667
Recall (Sensitivity): 0.8000
Specificity: 0.2000


In [ ]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO

# reproducibility
np.random.seed(42)
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# داده‌ی ساختگی: ۲۷ نمونه، ۵۰۰ ویژگی، ۲ کلاس
X, y = make_classification(n_samples=27, n_features=500, n_classes=2, n_informative=20, random_state=42)

# نرمال‌سازی
scaler = StandardScaler()
X = scaler.fit_transform(X)

# چون تعداد نمونه کم هست، یک split ساده انجام می‌دیم
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# فضای action: انتخاب از بین [8, 16, 32, 64]
layer_sizes = [8, 16, 32, 64, 128, 256, 512]

class NASEnv(gym.Env):
    def __init__(self):
        super(NASEnv, self).__init__()
        self.action_space = spaces.Discrete(len(layer_sizes))
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.state = np.array([0.0], dtype=np.float32)

    def reset(self, seed=None, options=None):
      super().reset(seed=seed)
      self.state = np.random.rand(1).astype(np.float32)  # تولید یک observation جدید مطابق با shape=(1,)
      return self.state, {}


    def step(self, action):
      hidden_dim = layer_sizes[action]
      acc = self._train_and_eval(hidden_dim)
      reward = acc
      terminated = True   # اپیزود در اینجا بعد از یک action تمام می‌شود
      truncated = False   # اینجا نیاز نداریم
      info = {"accuracy": acc}
      self.state = np.array([acc], dtype=np.float32)  # برای مثال، دقت را به عنوان state بروز می‌کنیم
      return self.state, reward, terminated, truncated, info


    def _train_and_eval(self, hidden_dim):
        model = nn.Sequential(
            nn.Linear(X_train.shape[1], hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        ).to(device)

        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.BCELoss()

        X_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

        model.train()
        for _ in range(10):  # چون داده کم هست، چند epoch کوتاه
            optimizer.zero_grad()
            preds = model(X_tensor)
            loss = criterion(preds, y_tensor)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy().flatten()
            preds = (preds >= 0.5).astype(int)
            acc = (preds == y_val).mean()
        return acc

# محیط
env = NASEnv()

# الگوریتم PPO
model = PPO('MlpPolicy', env, verbose=1)

# آموزش
model.learn(total_timesteps=1000)

# تست
obs, _ = env.reset()
action, _ = model.predict(obs)
print(f"\n✅ انتخاب کنترلر (action): {action} → hidden_dim={layer_sizes[action]}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.497    |
| time/              |          |
|    fps             | 51       |
|    iterations      | 1        |
|    time_elapsed    | 39       |
|    total_timesteps | 2048     |
---------------------------------

✅ انتخاب کنترلر (action): 2 → hidden_dim=32


In [ ]:
obs, _ = env.reset()
action, _ = model.predict(obs)
next_state, reward, terminated, truncated, info = env.step(action)
print(f"ریوارد (دقت مدل) برای action={action} ({layer_sizes[action]}): {reward}")

ریوارد (دقت مدل) برای action=2 (32): 0.4444444444444444
